In [1]:
from util import load_user_data
import pandas as pd
import numpy as np

In [2]:
valid_users, tp_data, _ = load_user_data(filename="../data/xailabdata_all.csv")
valid_users_2, tp_data_2, _ = load_user_data(filename="../data/xailabdata_llm_agent.csv")

valid_users = valid_users | valid_users_2
tp_data.update(tp_data_2)
variable_dict = {}
variable_dict["condition"] = []
variable_dict["engagement"] = []
dimensions = ["Reliability/Competence", "Understandability/Predictability", "Trust in Automation"]
covariates = ["ATI", "mlbackground"]
all_conditions = ["control", "dashboard", "chatxai", "chatxaiboost", "chatxaiAuto"]
condition_dict = {}
for dimension in dimensions:
    variable_dict[dimension] = []
for condition in all_conditions:
    condition_dict[condition] = {}
    for dimension in dimensions:
        condition_dict[condition][dimension] = []
for dimension in ["Propensity to Trust", "Familiarity", "ATI", "mlbackground"]:
    variable_dict[dimension] = []
print(condition_dict)
for user in valid_users:
    tp_condition = tp_data[user]["condition"]
    # if tp_condition == "control":
    # 	continue
    trust = tp_data[user]["Trust_in_automation"]
    variable_dict["condition"].append(tp_condition)
    user_engagement = tp_data[user]["user_engagement_scale"]
    variable_dict["engagement"].append(user_engagement)
    for dimension in dimensions:
        variable_dict[dimension].append(trust[dimension])
        condition_dict[tp_condition][dimension].append(trust[dimension])
    for dimension in ["Propensity to Trust", "Familiarity"]:
        variable_dict[dimension].append(trust[dimension])
    for dimension in ["ATI", "mlbackground"]:
        variable_dict[dimension].append(tp_data[user][dimension])

df = pd.DataFrame(variable_dict)
print(df.shape)

245 valid participants
{'control': 61, 'dashboard': 61, 'chatxai': 62, 'chatxaiboost': 61, 'chatxaiAuto': 0}
91 participants blindly rely on AI advice
{'control': 8, 'dashboard': 27, 'chatxai': 32, 'chatxaiboost': 24, 'chatxaiAuto': 0}
61 valid participants
{'control': 0, 'dashboard': 0, 'chatxai': 0, 'chatxaiboost': 0, 'chatxaiAuto': 61}
25 participants blindly rely on AI advice
{'control': 0, 'dashboard': 0, 'chatxai': 0, 'chatxaiboost': 0, 'chatxaiAuto': 25}
{'control': {'Reliability/Competence': [], 'Understandability/Predictability': [], 'Trust in Automation': []}, 'dashboard': {'Reliability/Competence': [], 'Understandability/Predictability': [], 'Trust in Automation': []}, 'chatxai': {'Reliability/Competence': [], 'Understandability/Predictability': [], 'Trust in Automation': []}, 'chatxaiboost': {'Reliability/Competence': [], 'Understandability/Predictability': [], 'Trust in Automation': []}, 'chatxaiAuto': {'Reliability/Competence': [], 'Understandability/Predictability': [], 

In [3]:
print("-" * 34)
from pingouin import ancova, anova
from statsmodels.stats.multicomp import pairwise_tukeyhsd
print("For all participants, compare with experimental conditions")
for dimension in dimensions:
    print(dimension)
    # aov = anova(dv=dimension, between='condition', data=df, effsize='n2')
    aov = ancova(dv=dimension, covar=["Propensity to Trust", "Familiarity", "ATI", "mlbackground"], between='condition', data=df, effsize='n2')
    print(aov.round(3))
    if aov.to_dict()['p-unc'][0] <= 0.05:
        tukey = pairwise_tukeyhsd(endog=df[dimension], groups=df['condition'], alpha=0.0125)
        print(tukey)
    for condition in all_conditions:
        print(condition, np.mean(condition_dict[condition][dimension]))
    print("-" * 17)

----------------------------------


/Users/ghe/opt/anaconda3/envs/agreement_phi/lib/python3.8/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.3, the latest is 0.5.5.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


For all participants, compare with experimental conditions
Reliability/Competence
                Source      SS   DF        F  p-unc     n2
0            condition   3.066    4    2.901  0.022  0.018
1  Propensity to Trust  84.303    1  319.098  0.000  0.505
2          Familiarity   0.244    1    0.923  0.337  0.001
3                  ATI   0.632    1    2.392  0.123  0.004
4         mlbackground   0.252    1    0.953  0.330  0.002
5             Residual  78.465  297      NaN    NaN    NaN
      Multiple Comparison of Means - Tukey HSD, FWER=0.01      
   group1       group2    meandiff p-adj   lower  upper  reject
---------------------------------------------------------------
     chatxai  chatxaiAuto  -0.2467 0.3683 -0.6842 0.1909  False
     chatxai chatxaiboost   0.1277 0.8818 -0.3099 0.5652  False
     chatxai      control  -0.3177  0.137 -0.7552 0.1199  False
     chatxai    dashboard   0.0156    1.0 -0.4219 0.4532  False
 chatxaiAuto chatxaiboost   0.3743 0.0507  -0.065 0.8136 

In [4]:
print("User Engagement")
aov = ancova(dv="engagement", covar=["Propensity to Trust", "Familiarity", "ATI", "mlbackground"], between='condition', data=df, effsize='n2')
print(aov.round(3))
if aov.to_dict()['p-unc'][0] <= 0.05:
    tukey = pairwise_tukeyhsd(endog=df["engagement"], groups=df['condition'], alpha=0.0125)
    print(tukey)

User Engagement
                Source       SS   DF       F  p-unc     n2
0            condition    2.656    4   1.957  0.101  0.020
1  Propensity to Trust   21.740    1  64.085  0.000  0.166
2          Familiarity    1.136    1   3.349  0.068  0.009
3                  ATI    4.469    1  13.173  0.000  0.034
4         mlbackground    0.484    1   1.427  0.233  0.004
5             Residual  100.755  297     NaN    NaN    NaN
